# TBD - Kaggle

## Authors
| **Name**              | **NIU**   |
|-----------------------|-----------|
| Arnau Muñoz Barrera   | 1665982   |
| José Ortín López      | 1667573   |


## Database

To access the source Database: [Link to Database](https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data)



## Import Libraries

In [20]:
# install required packages in the notebook environment
%pip install pandas numpy seaborn

import pandas as pd
import numpy as np
import seaborn as sns

Note: you may need to restart the kernel to use updated packages.


## Review Initial Structure and Data

In [21]:
df = pd.read_csv('data/vehicles-dataset.csv', engine='python', on_bad_lines='skip')

print(df.head(10).to_string())

           id                                                                                        url  region                     region_url  price  year manufacturer                  model  condition    cylinders    fuel odometer title_status transmission                VIN drive         size    type paint_color                                                            image_url                                                                                                                                                                                                description  county state      lat     long              posting_date;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
0  7306114308         https://albany.craigslist.org/cto/d/amsterdam-1961-chevy-10-apache/7306114308.html  albany  https://albany.craigslist.org  22000  1961    chevrolet                   c/10  excellent  8 cylinders     gas    85000        clean    automatic                NaN   rwd    full-size  pickup      cu

##### Determine columns & their types

In [22]:
print("[INFO] Dataset columns & their types: \n", df.dtypes)

[INFO] Dataset columns & their types: 
 id                                                      object
url                                                     object
region                                                  object
region_url                                              object
price                                                   object
year                                                    object
manufacturer                                            object
model                                                   object
condition                                               object
cylinders                                               object
fuel                                                    object
odometer                                                object
title_status                                            object
transmission                                            object
VIN                                                     object
drive          

##### Determine quantity of NaNs

In [23]:
# Auxiliar function to display missing values
def has_nans(df):
    return df.isna().sum().sum() > 0

# Auxiliar function to display percentage of missing values per column
def get_percentage_nan_per_column(df):
    return ((df.isna().sum().sort_values()) / (len(df) * 100))

print("[INFO] Does the dataset have missing values?", "Yes" if has_nans(df) else "No")

if has_nans(df):
    print("\n [INFO] Percentage of missing values per column: \n", get_percentage_nan_per_column(df))
else:
    print("\n [INFO] No missing values detected in the dataset.")


[INFO] Does the dataset have missing values? Yes

 [INFO] Percentage of missing values per column: 
 id                                                     0.000000
url                                                    0.001375
region                                                 0.001747
region_url                                             0.001747
price                                                  0.001747
year                                                   0.001765
fuel                                                   0.001765
odometer                                               0.001765
transmission                                           0.001765
title_status                                           0.001765
posting_date;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;    0.001846
state                                                  0.001846
image_url                                              0.001863
description                                            0.001863
lat

### Conclusions

As we can see in previous results, the main initial factors that should be revised & look into are:

| **Problem description**                                | **Proposed Solution**      | **Affected fields**      |
|--------------------------------------------------------|----------------------------|-------------------------|
| Field that needs format clean-up                       | Field modifications        | **posting_date**        |
| Field with most NaN values                             | Erase column               | **county**              |
| Fields with different types or inconsistent ranges     | Normalization              | **price**, **odometer** |
| Irrelevant fields                                      | Erase column               | **TBD**                 |

